In [ ]:
import pathlib
import re
import collections
import functools
import difflib
import operator
import warnings
import pickle

In [ ]:
%run utils.py
#cache = ApprovedMatches(datadir / "approved.pickle")

# Fix the birthplace field

## Part one: replace "i sognet"

In [ ]:
with AllEntries() as dataSet:
    for fn, year, entries in dataSet.getEntries():
        print(year)
        lines = [commonHeader + "\n"]
        for entry in entries:
            # prøv at fikse fødesteder
            if "her i sognet" in entry.fødested or \
               "heri sognet" in entry.fødested or \
                entry.fødested.replace(".", "") in ("i sognet", "her i s"):        
                # det kan være, der står noget om placering i parentes; brug det
                match1 = re.search(r"(her ?)?i sognet (\{|\[|\()(?P<place>.*?)(\]|\)|\})", entry.fødested)
                # det kan også være, der står noget efter et komma
                match2 = re.search(r"(her ?)? i sognet, (<P<place>.*)", entry.fødested)
                if match1:
                    new = match1.group("place")
                elif match2:
                    new = match2.group("place")
                else:
                    new = entry.sogn
                entry = entry._replace(fødested=new)
            lines.append(entry.toRow() + "\n")
        with fn.open("w", encoding="UTF-8") as fd:
            fd.writelines(lines)

In [ ]:
re.search(r"(her ?)?i sognet (\[|\()(?P<place>.*?)(\]|\))", "her i sognet [tommerup)").group("place")

In [ ]:
# for 1845 som eksempel
sorted((a,b) for (a,b) in changes if a not in ("her i sognet", "heri sognet", "i sognet"))

## Part two: more of the same

Let's see what's missing; the GPS lookup efforts showed a lot of crud.

**TODO**

In [ ]:
re_sogn_amt = re.compile(r"(.+) \[?sogn\]?,? ?(.+) \[?amt\]?")
re_do_sogn = re.compile(r"do \[(.+)\]")
_trans = {
    "kjøbenhavn": "københavn",
    "kiøbenhavn": "københavn",
    "kbhvn": "københavn",
    "sverrig": "sverige",
    "sverige": "sverige",
    "aarhus": "århus",
    "aarhuus": "århus",
    "rønne købstad - bornholms amt": "rønne",
    "kbhv": "københavn"
}
def normalize_more(place):
    if place in ("do", "do.", "her", "født i sognet", "dito", "sognet", "h. i s."):
        return None
    elif place in _trans:
        return _trans[place]
    match = re_sogn_amt.match(place)
    if match:
        return match.group(1)
    match = re_do_sogn.match(place)
    if match:
        return match.group(1)
    return place
